# Third Environement -  New Models 
## LightFM & Hybrid Model 
_Mariem Kachouri & Dinara Veshchezerova_

In [25]:
import warnings
warnings.filterwarnings('ignore')

from keras.models import model_from_json
import pandas as pd
import numpy as np
import gc
import requests 
from time import sleep
import json

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from lightfm import LightFM
from scipy.sparse import csr_matrix

In [5]:
## Retrieving useful functions
%run useful_functions.py
from useful_functions import *

In [29]:
user_id = 'VWW01S0D18NSKR55Q43F'
base_url ='http://35.180.178.243/'
url_reset=base_url+"reset"
url_predict = base_url+'predict'
params = {'user_id':user_id}
r = requests.get(url=url_reset,params=params) # get history of rating
data = r.json()

nb_users = data['nb_users'] # 100
nb_items = data['nb_items'] # 30

action_history = data['action_history']
state_history = data['state_history']
rewards_history = data['rewards_history']
next_state = data['next_state']

print('Number of users {}'.format(nb_users))
print('Number of items {}'.format(nb_items))
print('Nb of items available in the next state to predict: {}'.format(len(next_state)))

Number of users 100
Number of items 30
Nb of items available in the next state to predict: 30


In [30]:
# Retreiving user ids from the state history 
users_ids = list(zip(*list(list(zip(*state_history))[0])))[0]
pos_rewards = compute_pos_rewards(rewards_history)
print('Number of positive rewards:', len(list(pos_rewards.keys())))

pos_data = create_pos_data(pos_rewards,state_history,action_history)
pos_data.head(10)

Number of positive rewards: 64


,user_id,item_id,feat_users,pos_items
0,17,6,"[1.2696990910337893, 3.0979481697006945]",[6]
1,8,4,"[1.1249476193135086, -0.47011565372995134]","[4, 29]"
2,62,10,"[0.49105398187036775, 0.05817643603919631]",[10]
3,87,14,"[1.2207907408660346, 1.5870424387775075]","[14, 8]"
4,20,23,"[0.8611309672954842, 0.790983181392787]",[23]
5,4,15,"[1.4249964516629343, 3.080292729484041]",[15]
6,11,21,"[-0.40169705477315554, 3.239932656359141]",[21]
7,12,13,"[3.803091919382575, 1.045864723714945]",[13]
8,90,24,"[0.8201466388952966, 1.4825018411552278]",[24]
9,96,13,"[2.962805092202511, 1.7480251388684769]","[13, 3]"


In [31]:
dict_users, dict_items = {}, {}
for i,j in enumerate(pos_data['user_id']):
    dict_users[j] = i
for i,j in enumerate(pos_data['item_id']):
    dict_items[j] = i
rows = [dict_users[i] for i in pos_data['user_id']]
columns = [dict_items[i] for i in pos_data['item_id']]

M,N = np.max(rows), np.max(columns)
c = csr_matrix((np.ones((len(pos_data))), (rows, columns)), shape=(M+1, N+1))

# Parameters for LightFM
nb_threads, nb_components, nb_epochs, alpha = 2, 30, 60, 1e-5

# Let's fit a WARP model: these generally have the best performance.
model_lightFM = LightFM(loss='warp',learning_schedule='adagrad',item_alpha=alpha,no_components=nb_components)

In [32]:
%time 
model_lightFM.fit(c, epochs=nb_epochs, num_threads=nb_threads)

Wall time: 0 ns


In [33]:
nb_iters = 1000
rewards = 0
nb_reward_pos=0
new_users =[]

for i in range(nb_iters):
    sleep(0.05) # sleep to let the API breathe and allow others to call requests

    list_items = np.asarray(list(list(zip(*next_state))[1]))  # available items 
    
    if next_state[0][0] in pos_data.user_id.unique().tolist():
        next_user = dict_users[next_state[0][0]]
        
    else:
        #predict items based on users' profile similarity 
        most_similar_user_id = compute_most_similar(state_history,next_state,pos_data) 
        next_user = dict_users[most_similar_user_id]

   
    predictions = model_lightFM.predict(user_ids=next_user,item_ids=list_items)
    recommended_item = np.argmax(predictions) # position item 

    params['recommended_item'] = recommended_item 
    r=requests.get(url=url_predict,params=params)
    d=r.json()
    reward= d['reward'] # previous reward for the recommended item predicted
    if reward > 0 : 
        nb_reward_pos+=1 
    else:
        new_users.append(next_state[0][0])
    
    print('user: %s |recommended item position: %s | recommended item id %s | reward: %s' % (next_user,params['recommended_item'],next_state[params['recommended_item']][1],d['reward']))
    next_state = d['state']
    
    rewards += reward
    
print('Average reward: ', rewards/nb_iters)
print('Percentage of positive rewards: ', 100*(nb_reward_pos/nb_iters), '%')

user: 31 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 51 |recommended item position: 2 | recommended item id 3 | reward: 21.205146075868637
user: 38 |recommended item position: 19 | recommended item id 19 | reward: 0
user: 7 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 38 |recommended item position: 19 | recommended item id 19 | reward: 54.90050915503077
user: 55 |recommended item position: 20 | recommended item id 20 | reward: 872.587002948303
user: 0 |recommended item position: 19 | recommended item id 20 | reward: 0
user: 5 |recommended item position: 18 | recommended item id 19 | reward: 54.90050915503077
user: 29 |recommended item position: 9 | recommended item id 10 | reward: 561.3527699550926
user: 58 |recommended item position: 20 | recommended item id 20 | reward: 0
user: 29 |recommended item position: 13 | recommended item id 15 | reward: 0
user: 27 |recommended item position: 15 | recommended item id 15 | reward: 0
us

user: 63 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 44 |recommended item position: 19 | recommended item id 19 | reward: 0
user: 25 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 41 |recommended item position: 13 | recommended item id 15 | reward: 0
user: 45 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 38 |recommended item position: 9 | recommended item id 10 | reward: 0
user: 60 |recommended item position: 15 | recommended item id 15 | reward: 714.4126986921515
user: 8 |recommended item position: 15 | recommended item id 15 | reward: 714.4126986921515
user: 11 |recommended item position: 18 | recommended item id 18 | reward: 736.6534350516102
user: 10 |recommended item position: 10 | recommended item id 10 | reward: 561.3527699550926
user: 30 |recommended item position: 19 | recommended item id 19 | reward: 54.90050915503077
user: 16 |recommended item position: 14 | recommended item id 15 | reward: 71

user: 42 |recommended item position: 14 | recommended item id 15 | reward: 0
user: 60 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 27 |recommended item position: 10 | recommended item id 10 | reward: 561.3527699550926
user: 25 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 0 |recommended item position: 20 | recommended item id 20 | reward: 0
user: 38 |recommended item position: 9 | recommended item id 10 | reward: 561.3527699550926
user: 7 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 5 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 62 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 60 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 34 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 44 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 34 |recommended item position: 3 | recommended i

user: 55 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 25 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 31 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 35 |recommended item position: 16 | recommended item id 18 | reward: 0
user: 38 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 59 |recommended item position: 15 | recommended item id 15 | reward: 714.4126986921515
user: 0 |recommended item position: 20 | recommended item id 20 | reward: 0
user: 19 |recommended item position: 18 | recommended item id 19 | reward: 0
user: 48 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 31 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 52 |recommended item position: 17 | recommended item id 18 | reward: 0
user: 0 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 31 |recommended item position: 18 | recommended item id 18

user: 59 |recommended item position: 17 | recommended item id 18 | reward: 0
user: 2 |recommended item position: 17 | recommended item id 20 | reward: 0
user: 62 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 37 |recommended item position: 9 | recommended item id 10 | reward: 0
user: 0 |recommended item position: 20 | recommended item id 20 | reward: 872.587002948303
user: 59 |recommended item position: 17 | recommended item id 18 | reward: 736.6534350516102
user: 29 |recommended item position: 15 | recommended item id 18 | reward: 0
user: 41 |recommended item position: 13 | recommended item id 15 | reward: 0
user: 7 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 31 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 35 |recommended item position: 16 | recommended item id 18 | reward: 0
user: 0 |recommended item position: 17 | recommended item id 19 | reward: 54.90050915503077
user: 7 |recommended item position

user: 30 |recommended item position: 17 | recommended item id 18 | reward: 0
user: 0 |recommended item position: 20 | recommended item id 20 | reward: 0
user: 25 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 55 |recommended item position: 14 | recommended item id 17 | reward: 301.17067890236535
user: 54 |recommended item position: 10 | recommended item id 10 | reward: 561.3527699550926
user: 60 |recommended item position: 9 | recommended item id 10 | reward: 0
user: 31 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 0 |recommended item position: 9 | recommended item id 10 | reward: 0
user: 7 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 58 |recommended item position: 20 | recommended item id 20 | reward: 0
user: 50 |recommended item position: 17 | recommended item id 18 | reward: 736.6534350516102
user: 25 |recommended item position: 15 | recommended item id 15 | reward: 714.4126986921515
user: 61 |recomm

user: 7 |recommended item position: 15 | recommended item id 20 | reward: 872.587002948303
user: 7 |recommended item position: 2 | recommended item id 2 | reward: 183.70129650388168
user: 7 |recommended item position: 8 | recommended item id 11 | reward: 0
user: 58 |recommended item position: 20 | recommended item id 20 | reward: 872.587002948303
user: 0 |recommended item position: 15 | recommended item id 15 | reward: 714.4126986921515
user: 55 |recommended item position: 3 | recommended item id 4 | reward: 677.510654686604
user: 0 |recommended item position: 18 | recommended item id 18 | reward: 736.6534350516102
user: 59 |recommended item position: 17 | recommended item id 18 | reward: 0
user: 43 |recommended item position: 9 | recommended item id 10 | reward: 561.3527699550926
user: 0 |recommended item position: 18 | recommended item id 19 | reward: 0
user: 31 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 50 |recommended item position: 17 | recommended i

user: 26 |recommended item position: 9 | recommended item id 10 | reward: 0
user: 44 |recommended item position: 14 | recommended item id 15 | reward: 0
user: 62 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 37 |recommended item position: 14 | recommended item id 15 | reward: 0
user: 38 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 15 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 16 |recommended item position: 9 | recommended item id 10 | reward: 0
user: 48 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 32 |recommended item position: 17 | recommended item id 18 | reward: 736.6534350516102
user: 35 |recommended item position: 16 | recommended item id 18 | reward: 0
user: 35 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 15 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 58 |recommended item position: 17 | recommended item id 

user: 58 |recommended item position: 14 | recommended item id 15 | reward: 714.4126986921515
user: 43 |recommended item position: 15 | recommended item id 19 | reward: 54.90050915503077
user: 59 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 54 |recommended item position: 9 | recommended item id 11 | reward: 0
user: 53 |recommended item position: 18 | recommended item id 20 | reward: 0
user: 0 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 16 |recommended item position: 9 | recommended item id 10 | reward: 561.3527699550926
user: 43 |recommended item position: 3 | recommended item id 3 | reward: 21.205146075868637
user: 54 |recommended item position: 9 | recommended item id 11 | reward: 0
user: 5 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 25 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 63 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 31 |recommended 

user: 57 |recommended item position: 3 | recommended item id 3 | reward: 0
user: 0 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 31 |recommended item position: 18 | recommended item id 18 | reward: 736.6534350516102
user: 32 |recommended item position: 14 | recommended item id 15 | reward: 0
user: 25 |recommended item position: 16 | recommended item id 18 | reward: 0
user: 63 |recommended item position: 18 | recommended item id 18 | reward: 0
user: 47 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 30 |recommended item position: 17 | recommended item id 18 | reward: 0
user: 63 |recommended item position: 10 | recommended item id 10 | reward: 0
user: 61 |recommended item position: 16 | recommended item id 19 | reward: 0
user: 37 |recommended item position: 16 | recommended item id 19 | reward: 0
user: 50 |recommended item position: 14 | recommended item id 15 | reward: 714.4126986921515
user: 44 |recommended item position: 18 | recom

__Interpretation:__<br> Trying another model based on WRAP loss which is short for Weighted Approximate-Rank Pairwise loss. This loss maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found (i.e. a negative sample has a better ranking than a positive one). We had at the first time, out of the 1000 steps, 25% of users actually bought the item recommeded. <br> 

It is mainly based on the idea of Matrix Factorization so it doesn't really take into consideration the cold start issue. For this reason, we thought about trying a __hybrid model.__ <br>

## Model Hybrid: LightFM + Covariates / Cold Start 

In [12]:
user_id = 'VWW01S0D18NSKR55Q43F'
base_url ='http://35.180.178.243/'
url_reset=base_url+"reset"
url_predict = base_url+'predict'
params = {'user_id':user_id}
r = requests.get(url=url_reset,params=params) # get history of rating
data = r.json()

nb_users = data['nb_users'] # 100
nb_items = data['nb_items'] # 30

action_history = data['action_history']
state_history = data['state_history']
rewards_history = data['rewards_history']
next_state = data['next_state']

# Retreiving user ids from the state history 
users_ids = list(zip(*list(list(zip(*state_history))[0])))[0]
pos_rewards = compute_pos_rewards(rewards_history)
print('Number of positive rewards:', len(list(pos_rewards.keys())))

pos_data = create_pos_data(pos_rewards,state_history,action_history)
pos_data.head(10)

Number of positive rewards: 55


,user_id,item_id,feat_users,pos_items
0,67,2,"[1.7619113741850052, 2.037321402446281]","[2, 19, 11]"
1,29,21,"[-0.013245172254577886, 2.170905423099816]","[21, 3]"
2,53,26,"[1.4588966686768803, 1.9131191456684173]",[26]
3,50,29,"[2.2628515972244725, -0.5941908641710214]",[29]
4,17,29,"[2.3540326507517495, 1.6106336283178608]","[29, 16]"
5,73,7,"[0.9692410829084999, 1.3842053074604979]","[7, 27]"
6,99,13,"[2.184958393926907, 1.2398871564872913]",[13]
7,42,22,"[1.919862030687747, 0.8869175094585589]",[22]
8,40,14,"[1.4133346073263153, -2.222292107336869]",[14]
9,17,16,"[2.3540326507517495, 1.6106336283178608]","[29, 16]"


In [13]:
## Preprocessign for model LightFM
dict_users, dict_items = {}, {}
for i,j in enumerate(pos_data['user_id']):
    dict_users[j] = i
for i,j in enumerate(pos_data['item_id']):
    dict_items[j] = i
rows = [dict_users[i] for i in pos_data['user_id']]
columns = [dict_items[i] for i in pos_data['item_id']]

M,N = np.max(rows), np.max(columns)
c = csr_matrix((np.ones((len(pos_data))), (rows, columns)), shape=(M+1, N+1))

### Fitting LightFM

In [14]:
# Parameters for LightFM
nb_threads, nb_components, nb_epochs, alpha = 2, 30, 60, 1e-5

# Let's fit a WARP model: these generally have the best performance.
model_lightFM = LightFM(loss='warp',learning_schedule='adagrad',item_alpha=alpha,no_components=nb_components)
model_lightFM.fit(c, epochs=nb_epochs, num_threads=nb_threads)

### Fitting Siamese Network with triplet loss

In [17]:
deep_match_model, deep_triplet_model = build_models_covariates_price(nb_users, nb_items, user_dim=32,
                                                                        item_dim= 15, n_hidden =2, hidden_size=64,
                                                                        dropout=0.1,l2_reg=0)

deep_triplet_model.compile(loss=identity_loss, optimizer='adam')
fake_y = np.ones_like(pos_data['user_id'])

n_epochs = 50

for i in range(n_epochs):
    # Sample new negatives to build different triplets at each epoch
    inputs = sample_quintuplets_price(pos_data,state_history, random_seed=i)
    
    # Fit the model incrementally by doing a single pass over the sampled quintuplets.
    deep_triplet_model.fit(inputs, fake_y, shuffle=True, 
                            batch_size=32,
                            epochs=1, 
                            verbose=2)

#     # Monitor the convergence of the model
#     test_auc = average_roc_auc(deep_match_model, pos_data, pos_data_test)
    print("Epoch %d/%d:"% (i + 1, n_epochs))

Epoch 1/1
 - 18s - loss: 1.0066
Epoch 1/50:
Epoch 1/1
 - 0s - loss: 0.9824
Epoch 2/50:
Epoch 1/1
 - 0s - loss: 0.9962
Epoch 3/50:
Epoch 1/1
 - 0s - loss: 0.9801
Epoch 4/50:
Epoch 1/1
 - 0s - loss: 0.9943
Epoch 5/50:
Epoch 1/1
 - 0s - loss: 0.9914
Epoch 6/50:
Epoch 1/1
 - 0s - loss: 0.9898
Epoch 7/50:
Epoch 1/1
 - 0s - loss: 0.9796
Epoch 8/50:
Epoch 1/1
 - 0s - loss: 0.9552
Epoch 9/50:
Epoch 1/1
 - 0s - loss: 0.8974
Epoch 10/50:
Epoch 1/1
 - 0s - loss: 0.9177
Epoch 11/50:
Epoch 1/1
 - 0s - loss: 0.8562
Epoch 12/50:
Epoch 1/1
 - 0s - loss: 0.9194
Epoch 13/50:
Epoch 1/1
 - 0s - loss: 0.8568
Epoch 14/50:
Epoch 1/1
 - 0s - loss: 0.9064
Epoch 15/50:
Epoch 1/1
 - 0s - loss: 0.8815
Epoch 16/50:
Epoch 1/1
 - 0s - loss: 0.8897
Epoch 17/50:
Epoch 1/1
 - 0s - loss: 0.9009
Epoch 18/50:
Epoch 1/1
 - 0s - loss: 0.8546
Epoch 19/50:
Epoch 1/1
 - 0s - loss: 0.8852
Epoch 20/50:
Epoch 1/1
 - 0s - loss: 0.8553
Epoch 21/50:
Epoch 1/1
 - 0s - loss: 0.9171
Epoch 22/50:
Epoch 1/1
 - 0s - loss: 0.8963
Epoch 23/

## Hybrid Model : Computing Preditcions 
    -Idea1: Comparing predictions of two models and take the most expensive recommended item in order to maximize the average reward. 
    - Idea2: Same intuition behind the first idea but, this time, we will recommend the least expensive one in order to maximize the number of times users actually buy the recommended items.

### IDEA 1

In [28]:
nb_iters = 1000
rewards = 0
nb_reward_pos=0

for i in range(nb_iters):
    sleep(0.05) # sleep to let the API breathe and allow others to call requests
    
    if next_state[0][0] in pos_data.user_id.unique().tolist():
        next_user = np.asarray([next_state[0][0] for i in range(len(next_state))])
        next_userLFM = dict_users[next_state[0][0]]
        list_feat_user = np.expand_dims(np.asarray([next_state[0][3:5] for i in range(len(next_state))]), axis=1)

    
    else:
        #predict items based on users' profile similarity 
        most_similar_user_id = compute_most_similar(state_history,next_state,pos_data) 
        next_user = np.asarray([most_similar_user_id for i in range(len(next_state))])
        next_userLFM = dict_users[most_similar_user_id]
        list_feat_user = list(pos_data.loc[pos_data.user_id==most_similar_user_id,'feat_users'])[0]
        list_feat_user = np.expand_dims(np.asarray([list_feat_user for i in range(len(next_state))]), axis=1)


    list_items = np.asarray(list(list(zip(*next_state))[1]))
    
    prices =list(list(zip(*next_state))[1])
    
    feat_items = [next_state[0][5:] for i in range(len(next_state))]
    price_mean = np.asarray(prices).mean()
    price_std = np.asarray(prices).std()
    prices_norm = [(price-price_mean)/price_std for price in prices ] 
    
    for i,feature in  enumerate(feat_items):
        feature.append(prices_norm[i])
    
    list_feat_items = np.expand_dims(np.asarray(feat_items), axis=1)
    
    
    
    predictions = deep_match_model.predict([next_user, list_items, list_feat_user, list_feat_items])
    
    predictionsLFM = model_lightFM.predict(user_ids=next_userLFM, item_ids=list_items)
    recommended_item = np.argmax(predictions)
    recommended_itemLFM = np.argmax(predictionsLFM)
    
    if recommended_item != recommended_itemLFM:
        if next_state[recommended_item][2] < next_state[recommended_itemLFM][2]:
            recommended_item = recommended_itemLFM
   
    params['recommended_item'] = recommended_item 
    r=requests.get(url=url_predict,params=params)
    d=r.json()
    reward= d['reward'] # previous reward for the recommended item predicted
    if reward > 0 : 
        nb_reward_pos+=1 
   
    
    print('user: %s |recommended item position: %s | recommended item id %s | reward: %s' % (next_user[0],params['recommended_item'],next_state[params['recommended_item']][1],d['reward']))
    next_state = d['state']
    
    rewards += reward
    
print('Average reward: ', rewards/nb_iters)
print('Percentage of positive rewards: ', 100*(nb_reward_pos/nb_iters), '%')

user: 19 |recommended item position: 6 | recommended item id 6 | reward: 0
user: 25 |recommended item position: 22 | recommended item id 23 | reward: 0
user: 77 |recommended item position: 14 | recommended item id 14 | reward: 866.5152610742175
user: 52 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 76 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 34 |recommended item position: 13 | recommended item id 14 | reward: 866.5152610742175
user: 8 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 71 |recommended item position: 16 | recommended item id 16 | reward: 784.1267703259481
user: 8 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 49 |recommended item position: 14 | recommended item id 14 | reward: 866.5152610742175
user: 84 |recommended item position: 12 | recommended item id 14 | reward: 0
user: 94 |recommended item position: 14 | recommended item id 14 | reward: 0
user: 71 |recommended 

user: 64 |recommended item position: 0 | recommended item id 0 | reward: 208.13459861630344
user: 75 |recommended item position: 22 | recommended item id 23 | reward: 0
user: 65 |recommended item position: 23 | recommended item id 23 | reward: 0
user: 84 |recommended item position: 12 | recommended item id 14 | reward: 0
user: 7 |recommended item position: 14 | recommended item id 14 | reward: 0
user: 83 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 94 |recommended item position: 14 | recommended item id 14 | reward: 866.5152610742175
user: 34 |recommended item position: 14 | recommended item id 14 | reward: 0
user: 30 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 77 |recommended item position: 14 | recommended item id 14 | reward: 0
user: 75 |recommended item position: 0 | recommended item id 0 | reward: 208.13459861630344
user: 19 |recommended item position: 6 | recommended item id 6 | reward: 0
user: 30 |recommended item position

user: 97 |recommended item position: 13 | recommended item id 16 | reward: 784.1267703259481
user: 97 |recommended item position: 19 | recommended item id 23 | reward: 0
user: 75 |recommended item position: 21 | recommended item id 23 | reward: 664.6034852160633
user: 97 |recommended item position: 6 | recommended item id 6 | reward: 0
user: 52 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 8 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 19 |recommended item position: 6 | recommended item id 6 | reward: 0
user: 64 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 64 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 81 |recommended item position: 14 | recommended item id 14 | reward: 0
user: 94 |recommended item position: 15 | recommended item id 16 | reward: 784.1267703259481
user: 69 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 94 |recommended item position: 15

user: 70 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 19 |recommended item position: 13 | recommended item id 14 | reward: 0
user: 76 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 75 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 11 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 79 |recommended item position: 0 | recommended item id 0 | reward: 208.13459861630344
user: 76 |recommended item position: 11 | recommended item id 12 | reward: 562.9331395972741
user: 24 |recommended item position: 21 | recommended item id 23 | reward: 664.6034852160633
user: 77 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 26 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 97 |recommended item position: 19 | recommended item id 23 | reward: 664.6034852160633
user: 19 |recommended item position: 6 | recommended item id 6 | reward: 0
user: 52 |recommended i

user: 8 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 75 |recommended item position: 15 | recommended item id 16 | reward: 784.1267703259481
user: 26 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 70 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 55 |recommended item position: 12 | recommended item id 14 | reward: 866.5152610742175
user: 77 |recommended item position: 20 | recommended item id 23 | reward: 664.6034852160633
user: 29 |recommended item position: 3 | recommended item id 5 | reward: 0
user: 95 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 65 |recommended item position: 21 | recommended item id 23 | reward: 0
user: 75 |recommended item position: 22 | recommended item id 23 | reward: 664.6034852160633
user: 52 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 19 |recommended item position: 14 | recommended item id 16 | reward: 0
user: 69 |recommended i

user: 64 |recommended item position: 22 | recommended item id 24 | reward: 0
user: 59 |recommended item position: 2 | recommended item id 2 | reward: 808.823600356676
user: 55 |recommended item position: 0 | recommended item id 0 | reward: 208.13459861630344
user: 8 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 75 |recommended item position: 1 | recommended item id 2 | reward: 808.823600356676
user: 7 |recommended item position: 15 | recommended item id 16 | reward: 784.1267703259481
user: 72 |recommended item position: 1 | recommended item id 2 | reward: 0
user: 51 |recommended item position: 14 | recommended item id 16 | reward: 784.1267703259481
user: 8 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 97 |recommended item position: 1 | recommended item id 2 | reward: 0
user: 7 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 7 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 84 |recommen

user: 86 |recommended item position: 2 | recommended item id 2 | reward: 808.823600356676
user: 84 |recommended item position: 19 | recommended item id 23 | reward: 0
user: 4 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 65 |recommended item position: 23 | recommended item id 23 | reward: 0
user: 7 |recommended item position: 0 | recommended item id 0 | reward: 208.13459861630344
user: 70 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 51 |recommended item position: 10 | recommended item id 12 | reward: 0
user: 71 |recommended item position: 13 | recommended item id 14 | reward: 0
user: 34 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 77 |recommended item position: 2 | recommended item id 2 | reward: 808.823600356676
user: 75 |recommended item position: 12 | recommended item id 14 | reward: 0
user: 26 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 30 |recommended item position: 11 |

user: 77 |recommended item position: 20 | recommended item id 23 | reward: 0
user: 95 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 64 |recommended item position: 18 | recommended item id 20 | reward: 428.90376170051474
user: 11 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 8 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 84 |recommended item position: 19 | recommended item id 23 | reward: 0
user: 51 |recommended item position: 10 | recommended item id 12 | reward: 562.9331395972741
user: 96 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 64 |recommended item position: 13 | recommended item id 18 | reward: 0
user: 77 |recommended item position: 0 | recommended item id 0 | reward: 208.13459861630344
user: 64 |recommended item position: 10 | recommended item id 12 | reward: 0
user: 34 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 11 |recommended item position: 

user: 8 |recommended item position: 2 | recommended item id 4 | reward: 0
user: 95 |recommended item position: 1 | recommended item id 2 | reward: 808.823600356676
user: 7 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 79 |recommended item position: 3 | recommended item id 5 | reward: 860.3972599009837
user: 53 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 69 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 19 |recommended item position: 13 | recommended item id 14 | reward: 0
user: 75 |recommended item position: 15 | recommended item id 16 | reward: 784.1267703259481
user: 33 |recommended item position: 19 | recommended item id 24 | reward: 551.1934443842406
user: 22 |recommended item position: 13 | recommended item id 15 | reward: 654.5096734428913
user: 30 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 79 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 94 |rec

user: 7 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 43 |recommended item position: 3 | recommended item id 5 | reward: 0
user: 7 |recommended item position: 15 | recommended item id 16 | reward: 784.1267703259481
user: 77 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 77 |recommended item position: 4 | recommended item id 7 | reward: 0
user: 97 |recommended item position: 13 | recommended item id 14 | reward: 0
user: 8 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 96 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 52 |recommended item position: 20 | recommended item id 23 | reward: 0
user: 19 |recommended item position: 13 | recommended item id 14 | reward: 866.5152610742175
user: 77 |recommended item position: 2 | recommended item id 2 | reward: 0
user: 77 |recommended item position: 2 | recommended item id 2 | reward: 808.823600356676
user: 8 |recommended item position: 3 | recomm

As expected we have a larger value for the average reward comapred to previous models in third Environment 1.1 notebook and about 28% of the times (out of 1000), users actually bought the item recommened even if maight be more expensive.

In [18]:
#### IDEA 2:

In [19]:
nb_iters = 1000
rewards = 0
nb_reward_pos=0

for i in range(nb_iters):
    sleep(0.05) # sleep to let the API breathe and allow others to call requests
    
    if next_state[0][0] in pos_data.user_id.unique().tolist():
        next_user = np.asarray([next_state[0][0] for i in range(len(next_state))])
        next_userLFM = dict_users[next_state[0][0]]
        list_feat_user = np.expand_dims(np.asarray([next_state[0][3:5] for i in range(len(next_state))]), axis=1)

    
    else:
        #predict items based on users' profile similarity 
        most_similar_user_id = compute_most_similar(state_history,next_state,pos_data) 
        next_user = np.asarray([most_similar_user_id for i in range(len(next_state))])
        next_userLFM = dict_users[most_similar_user_id]
        list_feat_user = list(pos_data.loc[pos_data.user_id==most_similar_user_id,'feat_users'])[0]
        list_feat_user = np.expand_dims(np.asarray([list_feat_user for i in range(len(next_state))]), axis=1)


    list_items = np.asarray(list(list(zip(*next_state))[1]))
    
    prices =list(list(zip(*next_state))[1])
    
    feat_items = [next_state[0][5:] for i in range(len(next_state))]
    price_mean = np.asarray(prices).mean()
    price_std = np.asarray(prices).std()
    prices_norm = [(price-price_mean)/price_std for price in prices ] 
    
    for i,feature in  enumerate(feat_items):
        feature.append(prices_norm[i])
    
    list_feat_items = np.expand_dims(np.asarray(feat_items), axis=1)
    
    
    
    predictions = deep_match_model.predict([next_user, list_items, list_feat_user, list_feat_items])
    
    predictionsLFM = model_lightFM.predict(user_ids=next_userLFM, item_ids=list_items)
    recommended_item = np.argmax(predictions)
    recommended_itemLFM = np.argmax(predictionsLFM)
    
    if recommended_item != recommended_itemLFM:
        if next_state[recommended_item][2] > next_state[recommended_itemLFM][2]:
            recommended_item = recommended_itemLFM
   
    params['recommended_item'] = recommended_item 
    r=requests.get(url=url_predict,params=params)
    d=r.json()
    reward= d['reward'] # previous reward for the recommended item predicted
    if reward > 0 : 
        nb_reward_pos+=1 
   
    
    print('user: %s |recommended item position: %s | recommended item id %s | reward: %s' % (next_user[0],params['recommended_item'],next_state[params['recommended_item']][1],d['reward']))
    next_state = d['state']
    
    rewards += reward
    
print('Average reward: ', rewards/nb_iters)
print('Percentage of positive rewards: ', 100*(nb_reward_pos/nb_iters), '%')

user: 76 |recommended item position: 25 | recommended item id 26 | reward: 0
user: 41 |recommended item position: 20 | recommended item id 22 | reward: 451.52812795955765
user: 50 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 12 |recommended item position: 8 | recommended item id 8 | reward: 187.37257951758906
user: 2 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 73 |recommended item position: 25 | recommended item id 26 | reward: 0
user: 97 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 61 |recommended item position: 14 | recommended item id 14 | reward: 292.2258340871097
user: 17 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 40 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 54 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 44 |recommended item position: 22 | recommended item id 22 | reward: 451.52812795955765
user: 73 |reco

user: 29 |recommended item position: 29 | recommended item id 29 | reward: 0
user: 40 |recommended item position: 21 | recommended item id 22 | reward: 0
user: 67 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 11 |recommended item position: 26 | recommended item id 26 | reward: 509.1202112183504
user: 1 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 78 |recommended item position: 15 | recommended item id 15 | reward: 49.367280570713746
user: 36 |recommended item position: 22 | recommended item id 22 | reward: 451.52812795955765
user: 39 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 81 |recommended item position: 22 | recommended item id 22 | reward: 0
user: 40 |recommended item position: 8 | recommended item id 8 | reward: 187.37257951758906
user: 29 |recommended item position: 27 | recommended item id 29 | reward: 0
user: 48 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 73 |recommende

user: 11 |recommended item position: 25 | recommended item id 26 | reward: 0
user: 32 |recommended item position: 20 | recommended item id 22 | reward: 451.52812795955765
user: 1 |recommended item position: 19 | recommended item id 22 | reward: 0
user: 12 |recommended item position: 8 | recommended item id 8 | reward: 187.37257951758906
user: 9 |recommended item position: 13 | recommended item id 14 | reward: 292.2258340871097
user: 58 |recommended item position: 14 | recommended item id 14 | reward: 292.2258340871097
user: 48 |recommended item position: 23 | recommended item id 26 | reward: 0
user: 36 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 29 |recommended item position: 29 | recommended item id 29 | reward: 529.5636546253476
user: 36 |recommended item position: 22 | recommended item id 22 | reward: 0
user: 7 |recommended item position: 0 | recommended item id 0 | reward: 72.33554367361117
user: 39 |recommended item position: 0 | recommended item id 0 |

user: 77 |recommended item position: 22 | recommended item id 22 | reward: 0
user: 1 |recommended item position: 21 | recommended item id 22 | reward: 0
user: 77 |recommended item position: 22 | recommended item id 22 | reward: 0
user: 61 |recommended item position: 23 | recommended item id 26 | reward: 0
user: 17 |recommended item position: 25 | recommended item id 26 | reward: 0
user: 58 |recommended item position: 14 | recommended item id 14 | reward: 292.2258340871097
user: 48 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 51 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 79 |recommended item position: 14 | recommended item id 15 | reward: 49.367280570713746
user: 12 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 97 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 40 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 36 |recommended item position: 0 | recommend

user: 22 |recommended item position: 8 | recommended item id 8 | reward: 187.37257951758906
user: 53 |recommended item position: 22 | recommended item id 22 | reward: 0
user: 51 |recommended item position: 15 | recommended item id 15 | reward: 49.367280570713746
user: 11 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 42 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 17 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 77 |recommended item position: 22 | recommended item id 22 | reward: 451.52812795955765
user: 36 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 30 |recommended item position: 24 | recommended item id 26 | reward: 0
user: 36 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 54 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 11 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 21 |recommended item positi

user: 12 |recommended item position: 8 | recommended item id 8 | reward: 187.37257951758906
user: 11 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 11 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 11 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 12 |recommended item position: 21 | recommended item id 22 | reward: 0
user: 12 |recommended item position: 21 | recommended item id 22 | reward: 0
user: 18 |recommended item position: 26 | recommended item id 26 | reward: 509.1202112183504
user: 39 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 40 |recommended item position: 21 | recommended item id 22 | reward: 0
user: 22 |recommended item position: 13 | recommended item id 15 | reward: 0
user: 29 |recommended item position: 20 | recommended item id 20 | reward: 0
user: 39 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 58 |recommended item position: 21 | recomme

user: 81 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 42 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 30 |recommended item position: 20 | recommended item id 22 | reward: 0
user: 48 |recommended item position: 13 | recommended item id 14 | reward: 0
user: 9 |recommended item position: 14 | recommended item id 16 | reward: 0
user: 73 |recommended item position: 21 | recommended item id 22 | reward: 451.52812795955765
user: 44 |recommended item position: 14 | recommended item id 16 | reward: 0
user: 48 |recommended item position: 13 | recommended item id 14 | reward: 0
user: 97 |recommended item position: 16 | recommended item id 16 | reward: 0
user: 77 |recommended item position: 14 | recommended item id 15 | reward: 0
user: 29 |recommended item position: 20 | recommended item id 20 | reward: 684.2300556457978
user: 51 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 41 |recommended item position: 20 | reco

user: 44 |recommended item position: 14 | recommended item id 16 | reward: 0
user: 79 |recommended item position: 15 | recommended item id 15 | reward: 49.367280570713746
user: 12 |recommended item position: 15 | recommended item id 16 | reward: 458.87805380340023
user: 44 |recommended item position: 14 | recommended item id 16 | reward: 0
user: 41 |recommended item position: 15 | recommended item id 20 | reward: 684.2300556457978
user: 12 |recommended item position: 21 | recommended item id 22 | reward: 451.52812795955765
user: 81 |recommended item position: 23 | recommended item id 26 | reward: 0
user: 1 |recommended item position: 6 | recommended item id 8 | reward: 187.37257951758906
user: 50 |recommended item position: 25 | recommended item id 26 | reward: 509.1202112183504
user: 50 |recommended item position: 22 | recommended item id 22 | reward: 0
user: 81 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 35 |recommended item position: 19 | recommended it

user: 35 |recommended item position: 7 | recommended item id 8 | reward: 0
user: 51 |recommended item position: 15 | recommended item id 15 | reward: 0
user: 3 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 44 |recommended item position: 14 | recommended item id 16 | reward: 0
user: 1 |recommended item position: 23 | recommended item id 29 | reward: 0
user: 9 |recommended item position: 7 | recommended item id 8 | reward: 0
user: 11 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 11 |recommended item position: 26 | recommended item id 26 | reward: 0
user: 99 |recommended item position: 8 | recommended item id 8 | reward: 0
user: 94 |recommended item position: 25 | recommended item id 26 | reward: 0
user: 61 |recommended item position: 15 | recommended item id 16 | reward: 0
user: 81 |recommended item position: 22 | recommended item id 22 | reward: 0
user: 42 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 53 |reco

user: 40 |recommended item position: 21 | recommended item id 22 | reward: 451.52812795955765
user: 40 |recommended item position: 24 | recommended item id 26 | reward: 0
user: 22 |recommended item position: 0 | recommended item id 0 | reward: 72.33554367361117
user: 30 |recommended item position: 20 | recommended item id 22 | reward: 0
user: 36 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 39 |recommended item position: 26 | recommended item id 29 | reward: 0
user: 11 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 67 |recommended item position: 0 | recommended item id 0 | reward: 0
user: 39 |recommended item position: 26 | recommended item id 29 | reward: 0
user: 1 |recommended item position: 7 | recommended item id 8 | reward: 0
user: 47 |recommended item position: 13 | recommended item id 15 | reward: 49.367280570713746
user: 50 |recommended item position: 21 | recommended item id 22 | reward: 0
user: 17 |recommended item position: 

As expected, we had more users who bought the recommended items by the model but the average reward isn't satisfactory. 

__Perpespectives__ : 
    - How many times did the model witch predictions and took into consideration the Light FM ones 
    - How many times did the switch in predictions work 
    - What are the tendencies in predictions for both models : always recommend the most expensive? least expensive? not sensitive to price? 
    